# Tokenization, POS Tagging, and Dependency Parsing

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential

import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import re



In [6]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/HateSpeechData/data_cleaned.csv')

Mounted at /content/drive


In [ ]:
df.head()

,label,clean_tweet
0,1,أتفهم العثماني وأتفهم الجدل الشعبي الحل حان وق...
1,0,افظل حاكم عربي
2,1,ماريانو راخوي خاص المغرب واسبانيا يشرحو للرأي...
3,2,سمحت لك نفسك ان تقارن اردوغان بالحمار بنكيران ...
4,1,مليلية رئيس حزب بوكس الإسباني اللي كيعادي بلاد...


In [4]:
# model for tokenization
nltk.download('punkt')
#algorithm used for classification tasks (NER)
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

### NLTK Word Tokenize

In [8]:
sentence = df['clean_tweet'][0]

In [9]:
print(nltk.word_tokenize(sentence))

['أتفهم', 'العثماني', 'وأتفهم', 'الجدل', 'الشعبي', 'الحل', 'حان', 'وقت', 'تنزيل', 'شامل', 'للجهوية', 'المتقدمة', 'وتحمل', 'المتخبين', 'الجهوين', 'لكامل', 'صلاحياتهم', 'لشرح', 'تفا']


### Part-of-Speech (POS)

In [ ]:
!pip install stanza
import stanza

In [ ]:
import stanza

# Load the Arabic pipeline for Stanza
stanza.download('ar')
nlp_stanza = stanza.Pipeline('ar', processors='tokenize,pos')

INFO:stanza:Downloading default packages for language: ar (Arabic) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ar (Arabic):
| Processor | Package     |
---------------------------
| tokenize  | padt        |
| mwt       | padt        |
| pos       | padt_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [ ]:
for sentence in df['clean_tweet'][:5]:
    # Tokenize the sentence using Stanza
    doc_stanza = nlp_stanza(sentence)

    # Collect all tokens
    tokens = [word.text for sent in doc_stanza.sentences for word in sent.tokens]

    reconstructed_sentence = " ".join(tokens)
    print("Reconstructed Sentence:", reconstructed_sentence)

    # Perform POS tagging for each sentence using Stanza
    doc_stanza_pos = nlp_stanza(reconstructed_sentence)

    # Display POS tagging results
    for sent in doc_stanza_pos.sentences:
        for word in sent.words:
            print(f"Word: {word.text}\t POS: {word.pos}")


Reconstructed Sentence: أتفهم العثماني وأتفهم الجدل الشعبي الحل حان وقت تنزيل شامل للجهوية المتقدمة وتحمل المتخبين الجهوين لكامل صلاحياتهم لشرح تفا
Word: أتف	 POS: VERB
Word: هم	 POS: PRON
Word: العثماني	 POS: ADJ
Word: و	 POS: CCONJ
Word: أتفهم	 POS: VERB
Word: الجدل	 POS: NOUN
Word: الشعبي	 POS: ADJ
Word: الحل	 POS: NOUN
Word: حان	 POS: VERB
Word: وقت	 POS: NOUN
Word: تنزيل	 POS: NOUN
Word: شامل	 POS: ADJ
Word: ل	 POS: ADP
Word: الجهوية	 POS: NOUN
Word: المتقدمة	 POS: ADJ
Word: و	 POS: CCONJ
Word: تحمل	 POS: VERB
Word: المتخبين	 POS: NOUN
Word: الجهوين	 POS: ADJ
Word: ل	 POS: ADP
Word: كامل	 POS: ADJ
Word: صلاحيات	 POS: NOUN
Word: هم	 POS: PRON
Word: ل	 POS: ADP
Word: شرح	 POS: NOUN
Word: تفا	 POS: X
Reconstructed Sentence: افظل حاكم عربي
Word: افظل	 POS: X
Word: حاكم	 POS: ADJ
Word: عربي	 POS: ADJ
Reconstructed Sentence: ماريانو راخوي خاص المغرب واسبانيا يشرحو للرأي العام وجميع المتدخلين بوضوح تفاصيل الاتفاق اللي توصلو ليه
Word: ماريانو	 POS: X
Word: راخوي	 POS: X
Word: خاص	 POS: AD

###Perform dependency parsing

In [ ]:
!python -m spacy download xx_ent_wiki_sm


In [ ]:
!pip install stanza spacy


This annotation provides information about the words in a sentence, including their parts of speech (POS), head, and dependency relation.

In [ ]:
import stanza

# Load the Arabic pipeline for Stanza
stanza.download('ar')
nlp_stanza = stanza.Pipeline('ar', processors='tokenize,pos,lemma,depparse')

for sentence in df['clean_tweet'][:5]:
    # Tokenize the sentence using Stanza
    doc_stanza = nlp_stanza(sentence)


    print("**********     Tweet :    ****************", sentence)
    # Perform POS tagging for each sentence using Stanza
    doc_stanza_pos = nlp_stanza(doc_stanza)

    for sent in doc_stanza_pos.sentences:
        # Display dependency parsing results
        for word in sent.words:
            print(f"Word: {word.text}\t POS: {word.pos}\t Head: {word.head}\t Deprel: {word.deprel}")




INFO:stanza:Downloading default packages for language: ar (Arabic) ...
INFO:stanza:File exists: /root/stanza_resources/ar/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


**********     Tweet :    **************** أتفهم العثماني وأتفهم الجدل الشعبي الحل حان وقت تنزيل شامل للجهوية المتقدمة وتحمل المتخبين الجهوين لكامل صلاحياتهم لشرح تفا
Word: أتف	 POS: VERB	 Head: 0	 Deprel: root
Word: هم	 POS: PRON	 Head: 1	 Deprel: obj
Word: العثماني	 POS: ADJ	 Head: 1	 Deprel: nsubj
Word: و	 POS: CCONJ	 Head: 5	 Deprel: cc
Word: أتفهم	 POS: VERB	 Head: 1	 Deprel: conj
Word: الجدل	 POS: NOUN	 Head: 5	 Deprel: obj
Word: الشعبي	 POS: ADJ	 Head: 6	 Deprel: amod
Word: الحل	 POS: NOUN	 Head: 1	 Deprel: obj
Word: حان	 POS: VERB	 Head: 1	 Deprel: conj
Word: وقت	 POS: NOUN	 Head: 9	 Deprel: obj
Word: تنزيل	 POS: NOUN	 Head: 10	 Deprel: nmod
Word: شامل	 POS: ADJ	 Head: 11	 Deprel: amod
Word: ل	 POS: ADP	 Head: 14	 Deprel: case
Word: الجهوية	 POS: NOUN	 Head: 11	 Deprel: nmod
Word: المتقدمة	 POS: ADJ	 Head: 14	 Deprel: amod
Word: و	 POS: CCONJ	 Head: 17	 Deprel: cc
Word: تحمل	 POS: VERB	 Head: 9	 Deprel: conj
Word: المتخبين	 POS: NOUN	 Head: 17	 Deprel: obj
Word: الجهوين	 POS: A

Word: ندم

POS: NOUN (Noun)

Head: 1 (The noun "ندم" is the object of the preposition "ب" at position 1.)

Deprel: obl:arg (The noun "ندم" is an oblique argument of the verb "تفووووو".)

###Dependency tree with NLTK

In [ ]:
import spacy
from nltk import Tree

# Load SpaCy model for English
nlp_spacy = spacy.load("en_core_web_sm")

# Tokenize using SpaCy
sentence = df['clean_tweet'][7]
doc_spacy = nlp_spacy(sentence)

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

# Visualize the syntactic tree for each sentence
for sent in doc_spacy.sents:
    tree = to_nltk_tree(sent.root)
    tree.pretty_print()


           بنكيران         
  ____________|_________    
 |           انا        |  
 |            |         |   
 |         العدالة     الله
 |      ______|_____    |   
أخبت متحزب   حزب       خلق 



In [ ]:
import pandas as pd
import stanza
import nltk
from nltk import Tree

# Tokenize using Stanza
sentence = df['clean_tweet'][0]
doc_stanza = nlp_stanza(sentence)

# Build a list of tuples with (word, head) for each word in the sentence
dependencies = [(word.text, word.head) for sentence in doc_stanza.sentences for word in sentence.words]

# Build a tree using nltk.Tree
tree_str = "(ROOT " + " ".join([f"({word} {head})" for word, head in dependencies]) + ")"
tree = Tree.fromstring(tree_str)


tree.pretty_print()


                                                                ROOT                                                                            
  _______________________________________________________________|____________________________________________________________________________   
أتف  هم العثماني  و  أتفهم الجدل الشعبي الحل حان وقت تنزيل شامل  ل   الجهوية المتقدمة  و  تحمل المتخبين الجهوين  ل  كامل صلاحيات  هم  ل  شرح تفا
 |   |     |      |    |     |     |     |    |   |    |    |    |      |       |      |   |      |        |     |   |      |     |   |   |   |  
 0   1     1      5    1     5     6     1    1   9    10   11   14     11      14     17  9      17       18    21  18     21    22  25  22  25



###dependency tree using Spacy

In [ ]:
import spacy
from spacy import displacy

sentence = df['clean_tweet'][0]

# tokenize  using Stanza
doc_stanza = nlp_stanza(sentence)


# Perform POS tagging
doc_stanza_pos = nlp_stanza(doc_stanza)
 using Stanza
sent = doc_stanza_pos.sentences[0]

for word in sent.words:
    print(f"Word: {word.text}\t POS: {word.pos}\t Head: {word.head}\t Deprel: {word.deprel}")

# Display the dependency tree
#displacy.render(doc_stanza_pos, style='dep', options={'distance': 90, 'color': 'lightblue', 'bg': '#ffffff'}, jupyter=True)
displacy.render([doc_stanza_pos], style='dep', options={'distance': 90, 'color': 'lightblue', 'bg': '#ffffff'}, jupyter=True)



Word: أتف	 POS: VERB	 Head: 0	 Deprel: root
Word: هم	 POS: PRON	 Head: 1	 Deprel: obj
Word: العثماني	 POS: ADJ	 Head: 1	 Deprel: nsubj
Word: و	 POS: CCONJ	 Head: 5	 Deprel: cc
Word: أتفهم	 POS: VERB	 Head: 1	 Deprel: conj
Word: الجدل	 POS: NOUN	 Head: 5	 Deprel: obj
Word: الشعبي	 POS: ADJ	 Head: 6	 Deprel: amod
Word: الحل	 POS: NOUN	 Head: 1	 Deprel: obj
Word: حان	 POS: VERB	 Head: 1	 Deprel: conj
Word: وقت	 POS: NOUN	 Head: 9	 Deprel: obj
Word: تنزيل	 POS: NOUN	 Head: 10	 Deprel: nmod
Word: شامل	 POS: ADJ	 Head: 11	 Deprel: amod
Word: ل	 POS: ADP	 Head: 14	 Deprel: case
Word: الجهوية	 POS: NOUN	 Head: 11	 Deprel: nmod
Word: المتقدمة	 POS: ADJ	 Head: 14	 Deprel: amod
Word: و	 POS: CCONJ	 Head: 17	 Deprel: cc
Word: تحمل	 POS: VERB	 Head: 9	 Deprel: conj
Word: المتخبين	 POS: NOUN	 Head: 17	 Deprel: obj
Word: الجهوين	 POS: ADJ	 Head: 18	 Deprel: amod
Word: ل	 POS: ADP	 Head: 21	 Deprel: case
Word: كامل	 POS: ADJ	 Head: 18	 Deprel: amod
Word: صلاحيات	 POS: NOUN	 Head: 21	 Deprel: nmod
Word

ValueError: [E096] Invalid object passed to displaCy: Can only visualize `Doc` or Span objects, or dicts if set to `manual=True`.

###dependency parsing tree using grammar inputs in nltk






In [ ]:
import nltk

sentence = "هادشي علاش حرم الله النميمة (توصال الهدرة) لا يدخل الجنة نمام وباقي كلشي غيوقف أمام الله الحساب الأكبر كل واحد غياخد حقه,"
tokens = nltk.word_tokenize(sentence)
your_arabic_grammar = nltk.CFG.fromstring("""
S -> V NP
V -> 'حرم' | 'يدخل' | 'غيوقف | 'غياخد
NP -> PRP N | DT N PP | DT N | ADJ N PP | DT NP
PRP -> 'الله' | 'هادشي'
N -> 'النميمة' | 'الجنة' | 'كلشي' | 'الحساب' | 'واحد' | 'حقه'
PP -> P NP
P -> 'علاش' | 'توصال' | 'الهدرة' | 'أمام' | '(' | ')' | ','
DT -> 'ال' | 'كل'
ADJ -> 'الأكبر'
""")

parser = nltk.ChartParser(your_arabic_grammar)
for tree in parser.parse(tokens):
    tree.pretty_print()


ValueError: Grammar does not cover some of the input words: "'لا', 'نمام', 'وباقي', 'غيوقف', 'غياخد'".